### Google Colab utils

In [ ]:
#!pip install keras-tuner

In [ ]:
# # memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#  process = psutil.Process(os.getpid())
#  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm() 

In [ ]:
#!kill -9 -1

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')
# %cd "/gdrive/My Drive/air-pollution"

### Modeling

In [1]:
import warnings
warnings.filterwarnings('ignore')

from models import *
from kerastuner.tuners import RandomSearch

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
/home/zafir/miniconda3/envs/tensorflow/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
train_encoder_input_data = np.load('./data/third-order/Centar/train_encoder_input_data.npy')
train_decoder_input_data = np.load('./data/third-order/Centar/train_decoder_input_data.npy')
train_decoder_target_data = np.load('./data/third-order/Centar/train_decoder_target_data.npy')

valid_encoder_input_data = np.load('./data/third-order/Centar/valid_encoder_input_data.npy')
valid_decoder_input_data = np.load('./data/third-order/Centar/valid_decoder_input_data.npy')
valid_decoder_target_data = np.load('./data/third-order/Centar/valid_decoder_target_data.npy')

test_encoder_input_data = np.load('./data/third-order/Centar/test_encoder_input_data.npy')
test_decoder_input_data = np.load('./data/third-order/Centar/test_decoder_input_data.npy')
test_decoder_target_data = np.load('./data/third-order/Centar/test_decoder_target_data.npy')

In [3]:
print(train_encoder_input_data.shape)
print(train_decoder_input_data.shape)
print(train_decoder_target_data.shape)

(67396, 24, 23)
(67396, 12, 21)
(67396, 12, 2)


In [4]:
Tx, encoder_input_dim = (train_encoder_input_data.shape[1], 
                         train_encoder_input_data.shape[2])
    
Ty, decoder_input_dim = (train_decoder_input_data.shape[1], 
                         train_decoder_input_data.shape[2])

# we are predicting the pollution only, leave out the mask
decoder_output_dim = 1  

In [5]:
batch_size = 64
epochs = 250
max_trials = 250
executions_per_trial = 1
patience = 20

### Plain

In [6]:
model_builder = PlainSeq2Seq(Tx, Ty, encoder_input_dim, 
                             decoder_input_dim, decoder_output_dim)

tuner = RandomSearch(model_builder,
                     objective='val_loss',
                     max_trials=max_trials,
                     executions_per_trial=executions_per_trial,
                     directory='local-keras-tuner/inattentive', 
                     project_name='plain')

INFO:tensorflow:Reloading Oracle from existing project local-keras-tuner/inattentive/plain/oracle.json


In [7]:
tuner.search(x=[train_encoder_input_data, 
                train_decoder_input_data], 
             y=train_decoder_target_data,
             validation_data=([
                valid_encoder_input_data,
                valid_decoder_input_data],
                valid_decoder_target_data),
             batch_size=batch_size,
             epochs=epochs,
             callbacks=[EarlyStopping(monitor='val_loss', 
                                      patience=patience, 
                                      verbose=1)])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 67396 samples, validate on 3388 samples
Epoch 1/250


27584/67396 [===========>..................] - ETA: 24:06 - loss: 0.65 - ETA: 8:20 - loss: 0.6948 - ETA: 4:20 - loss: 0.660 - ETA: 3:24 - loss: 0.593 - ETA: 2:50 - loss: 0.574 - ETA: 2:27 - loss: 0.542 - ETA: 2:02 - loss: 0.508 - ETA: 1:51 - loss: 0.495 - ETA: 1:37 - loss: 0.464 - ETA: 1:32 - loss: 0.443 - ETA: 1:26 - loss: 0.431 - ETA: 1:19 - loss: 0.410 - ETA: 1:13 - loss: 0.395 - ETA: 1:08 - loss: 0.378 - ETA: 1:04 - loss: 0.365 - ETA: 1:02 - loss: 0.362 - ETA: 1:00 - loss: 0.359 - ETA: 59s - loss: 0.352 - ETA: 57s - loss: 0.34 - ETA: 56s - loss: 0.34 - ETA: 54s - loss: 0.33 - ETA: 53s - loss: 0.32 - ETA: 52s - loss: 0.32 - ETA: 50s - loss: 0.31 - ETA: 48s - loss: 0.31 - ETA: 46s - loss: 0.30 - ETA: 45s - loss: 0.30 - ETA: 44s - loss: 0.29 - ETA: 43s - loss: 0.29 - ETA: 42s - loss: 0.29 - ETA: 42s - loss: 0.29 - ETA: 42s - loss: 0.28 - ETA: 42s - loss: 0.28 - ETA: 41s - loss: 0.28 - ETA: 41s - loss: 0.28 - ETA: 41s - loss: 0.28 - ETA: 40s - loss: 0.27 - ETA: 40s - loss: 0.27 - ETA: 

KeyboardInterrupt: 

In [ ]:
tuner.results_summary()

### Stacked

In [8]:
model_builder = StackedSeq2Seq(Tx, Ty, encoder_input_dim, 
                             decoder_input_dim, decoder_output_dim)

tuner = RandomSearch(model_builder,
                     objective='val_loss',
                     max_trials=max_trials,
                     executions_per_trial=executions_per_trial,
                     directory='local-keras-tuner/inattentive', 
                     project_name='stacked')

In [9]:
tuner.search(x=[train_encoder_input_data, 
                train_decoder_input_data], 
             y=train_decoder_target_data,
             validation_data=([
                valid_encoder_input_data,
                valid_decoder_input_data],
                valid_decoder_target_data),
             batch_size=batch_size,
             epochs=epochs,
             callbacks=[EarlyStopping(monitor='val_loss', 
                                      patience=patience, 
                                      verbose=1)])

Train on 67396 samples, validate on 3388 samples
Epoch 1/250
 2816/67396 [>.............................] - ETA: 24:41 - loss: 0.94 - ETA: 13:06 - loss: 0.70 - ETA: 9:21 - loss: 0.6023 - ETA: 7:24 - loss: 0.537 - ETA: 6:14 - loss: 0.482 - ETA: 5:26 - loss: 0.453 - ETA: 4:55 - loss: 0.433 - ETA: 4:41 - loss: 0.412 - ETA: 4:21 - loss: 0.395 - ETA: 4:04 - loss: 0.371 - ETA: 3:52 - loss: 0.354 - ETA: 3:42 - loss: 0.336 - ETA: 3:31 - loss: 0.327 - ETA: 3:23 - loss: 0.315 - ETA: 3:18 - loss: 0.308 - ETA: 3:11 - loss: 0.296 - ETA: 3:07 - loss: 0.286 - ETA: 3:02 - loss: 0.277 - ETA: 2:57 - loss: 0.269 - ETA: 2:53 - loss: 0.261 - ETA: 2:50 - loss: 0.255 - ETA: 2:49 - loss: 0.248 - ETA: 2:46 - loss: 0.244 - ETA: 2:43 - loss: 0.237 - ETA: 2:41 - loss: 0.233 - ETA: 2:38 - loss: 0.228 - ETA: 2:37 - loss: 0.223 - ETA: 2:34 - loss: 0.219 - ETA: 2:31 - loss: 0.217 - ETA: 2:29 - loss: 0.214 - ETA: 2:28 - loss: 0.211 - ETA: 2:26 - loss: 0.206 - ETA: 2:25 - loss: 0.203 - ETA: 2:24 - loss: 0.200 - ETA: 2:

KeyboardInterrupt: 

In [ ]:
tuner.results_summary()

### BiStacked

In [ ]:
model_builder = BiStackedSeq2Seq(Tx, Ty, encoder_input_dim, 
                             decoder_input_dim, decoder_output_dim)

tuner = RandomSearch(model_builder,
                     objective='val_loss',
                     max_trials=max_trials,
                     executions_per_trial=executions_per_trial,
                     directory='local-keras-tuner/inattentive', 
                     project_name='bistacked')

In [ ]:
tuner.search(x=[train_encoder_input_data, 
                train_decoder_input_data], 
             y=train_decoder_target_data,
             validation_data=([
                valid_encoder_input_data,
                valid_decoder_input_data],
                valid_decoder_target_data),
             batch_size=batch_size,
             epochs=epochs,
             callbacks=[EarlyStopping(monitor='val_loss', 
                                      patience=patience, 
                                      verbose=1)])

In [ ]:
tuner.results_summary()